In [1]:
import pandas as pd
import pickle
import numpy as np
import re
import pymystem3
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, RandomizedSearchCV, StratifiedKFold
from scipy.stats import lognorm as sp_lognorm
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

In [2]:
# nltk.download('stopwords')
# nltk.download('wordnet')

## Read

In [3]:
train = pd.read_csv('train.csv', sep=';', index_col='id')

In [4]:
train.head()

,text,target
id,,
0,<p>В молодой и дружный коллектив динамично раз...,2
1,<p><strong>Обязанности: </strong>- Прием входя...,2
2,<p><strong>Федеральная сеть аптек </strong>в с...,1
3,<strong>Обязанности:</strong> <ul> <li> <p>раз...,0
4,<p>Производителю сантехники тм Domani Spa треб...,5


In [5]:
train.shape

(31063, 2)

In [6]:
train['target'].value_counts()

2    7586
0    5832
5    5674
3    4659
4    4259
1    3053
Name: target, dtype: int64

In [120]:
test = pd.read_csv('test.csv', sep=';', index_col='id')

In [121]:
test.head()

,text
id,
31063,<p><strong>В крупную компанию по организации и...
31064,<p><strong>Обязанности:</strong></p> <ul> <li>...
31065,<p> </p> <p><strong>Обязанности:</strong></p> ...
31066,<p><strong>Обязанности:</strong></p> <ul> <li>...
31067,<p><strong>Вакансия СРОЧНАЯ!</strong></p> <p><...


In [8]:
test.shape

(31064, 1)

## Data Preproc

In [9]:
def text_to_wordlist(text):
    text_wo_tags = re.sub("<[^>]*>", "", text.lower())
    text = re.sub('[^a-zA-Zа-яА-ЯёЁ]', ' ', text_wo_tags)
    words = text.lower().strip().split()
    return words

In [10]:
def clean(words, stopWords):
    new_words = [word for word in words if (word not in stopWords) and len(word) > 2]
    return new_words

In [11]:
mystem = pymystem3.Mystem()
wordnet_lemmatizer = WordNetLemmatizer()

def clean_v2(words):    
    new_words = [mystem.lemmatize(x)[0] for x in words]
    return new_words

In [12]:
def clean_v3(words):    
    new_words = [wordnet_lemmatizer.lemmatize(x, pos=wordnet.VERB) for x in words]
    return new_words

In [13]:
def preproc_data(df, stopWords):
    
    temp = df.copy()
    temp['norm_test'] = temp['text'].apply(lambda x: text_to_wordlist(x))
    print('v1 done')
    temp['norm_test_v2'] = temp['norm_test'].apply(lambda x: clean(x, stopWords))
    print('v2 done')
    temp['norm_test_v3'] = temp['norm_test_v2'].apply(lambda x: clean_v2(x))
    print('v3 done')
    temp['norm_test_v4'] = temp['norm_test_v3'].apply(lambda x: clean_v3(x))
    print('v4 done')
    temp['joined_text'] = temp['norm_test_v4'].apply(lambda x: ' '.join(x))
    joined_text = temp['joined_text'].tolist()
    
    return joined_text

In [14]:
%%time

stopWords = stopwords.words(['russian', 'english'])
train_joined_text = preproc_data(train, stopWords)

v1 done
v2 done
v3 done
v4 done
CPU times: user 2min 1s, sys: 37.5 s, total: 2min 39s
Wall time: 6min 1s


In [15]:
%%time
test_joined_text = preproc_data(test, stopWords)

v1 done
v2 done
v3 done
v4 done
CPU times: user 2min 2s, sys: 37.7 s, total: 2min 40s
Wall time: 6min 2s


In [16]:
prepr_train = pd.DataFrame(train_joined_text, columns=['text'])
prepr_train['target'] = train['target']

In [17]:
prepr_test = pd.DataFrame(test_joined_text, columns=['text'])

In [18]:
prepr_train.to_csv('prepr_train.csv', index=False)

In [19]:
prepr_test.to_csv('prepr_test.csv', index=False)

## Read preproc data

In [3]:
prepr_train = pd.read_csv('prepr_train.csv')
prepr_train.head()

,text,target
0,молодой дружный коллектив динамично развиватьс...,2
1,обязанность прием входить звонок заявка обрабо...,2
2,федеральный сеть аптека связь активный расшире...,1
3,обязанность разработка проектный рабочий докум...,0
4,производитель сантехник domani spa требоваться...,5


In [4]:
X_test = pd.read_csv('prepr_test.csv')
X_test.head()

,text
0,крупный компания организация приготовление кор...
1,обязанность обеспечение необходимый функционал...
2,обязанность отгрузка прием товар склад склад к...
3,обязанность приготовление холодный горячий блю...
4,вакансия срочный внимание просьба подробно изу...


In [5]:
X = prepr_train.drop('target', axis=1)
y = prepr_train.target

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)

In [7]:
train_list = X_train['text'].tolist()
valid_list = X_valid['text'].tolist()
test_list = X_test['text'].tolist()

In [9]:
text_list = train_list + valid_list + test_list

## Impr

In [7]:
train_0 = prepr_train[prepr_train.target==0]['text'].tolist()
train_1 = prepr_train[prepr_train.target==1]['text'].tolist()
train_2 = prepr_train[prepr_train.target==2]['text'].tolist()
train_3 = prepr_train[prepr_train.target==3]['text'].tolist()
train_4 = prepr_train[prepr_train.target==4]['text'].tolist()
train_5 = prepr_train[prepr_train.target==5]['text'].tolist()

In [10]:
tfidf = TfidfVectorizer(max_features=10000)
tfidf.fit_transform(text_list)

<62127x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 4825154 stored elements in Compressed Sparse Row format>

In [11]:
tfidf_0 = tfidf.transform(train_0)
tfidf_1 = tfidf.transform(train_1)
tfidf_2 = tfidf.transform(train_2)
tfidf_3 = tfidf.transform(train_3)
tfidf_4 = tfidf.transform(train_4)
tfidf_5 = tfidf.transform(train_5)

In [12]:
def top_words(tfidf_x):
    kk = pd.DataFrame(tfidf_x.todense(), columns=tfidf.get_feature_names())
    words = [x[0] for x in sorted(zip(kk.columns, kk.sum()), key=lambda x: x[1], reverse=True)[:100]]
    return words

words0 = top_words(tfidf_0)
words1 = top_words(tfidf_1)
words2 = top_words(tfidf_2)
words3 = top_words(tfidf_3)
words4 = top_words(tfidf_4)
words5 = top_words(tfidf_5)

In [13]:
all_words = words0 + words1 + words2 + words3 + words4 + words5

In [15]:
unique_words = []
bad_words = []
for w in all_words:
    if w in unique_words:
        bad_words.append(w)
    else:
        unique_words.append(w)

In [19]:
cl_words0 = [x for x in words0 if x not in set(bad_words)]
cl_words0

['объект',
 'строительный',
 'строительство',
 'проектный',
 'материал',
 'ремонт',
 'производство',
 'autocad',
 'проектирование',
 'исполнительный',
 'выполнять',
 'монтаж',
 'недвижимость',
 'сметный',
 'монтажный',
 'инженерный',
 'чертеж',
 'эксплуатация',
 'нормативный',
 'конструкция',
 'здание',
 'согласование',
 'раздел',
 'инженер',
 'производственный',
 'смета',
 'охрана',
 'проживание',
 'безопасность',
 'должность',
 'надзор']

In [20]:
cl_words1 = [x for x in words1 if x not in set(bad_words)]
cl_words1

['сертификат',
 'пациент',
 'врач',
 'специальность',
 'действующий',
 'клиника',
 'фармацевтический',
 'аптека',
 'лечение',
 'лекарственный',
 'препарат',
 'помощь',
 'средний',
 'лечебный',
 'диагностика',
 'дело',
 'исследование',
 'оказание',
 'больной',
 'кабинет',
 'амбулаторный',
 'сестринский',
 'территория',
 'диагностический',
 'ребенок',
 'процедура',
 'назначение',
 'аптечный',
 'социальный',
 'учреждение',
 'консультация',
 'осмотр',
 'приветствоваться',
 'мероприятие',
 'план',
 'квалификация']

In [21]:
cl_words2 = [x for x in words2 if x not in set(bad_words)]
cl_words2

['задача',
 'поддержка',
 'тестирование',
 'приложение',
 'понимание',
 'уровень',
 'настройка',
 'язык',
 'бизнес',
 'sql',
 'разработчик',
 'программный',
 'новый',
 'администрирование',
 'технология',
 'участие',
 'процесс',
 'информационный',
 'который',
 'сервис',
 'платформа',
 'сайт',
 'принцип',
 'код',
 'база',
 'английский',
 'программирование',
 'внедрение',
 'написание',
 'сетевой',
 'интересный',
 'конфигурация',
 'предлагать',
 'плюс',
 'создание',
 'мобильный',
 'хороший',
 'свой',
 'java',
 'сопровождение',
 'сервер',
 'дмс',
 'windows',
 'доработка',
 'web',
 'другой',
 'автоматизация',
 'php']

In [22]:
cl_words3 = [x for x in words3 if x not in set(bad_words)]
cl_words3

['бухгалтерский',
 'налоговый',
 'бухгалтер',
 'первичный',
 'отчетность',
 'финансовый',
 'бухгалтерия',
 'счет',
 'экономический',
 'сверка',
 'формирование',
 'отчет',
 'банк',
 'операция',
 'предприятие',
 'excel',
 'поставщик',
 'главный',
 'законодательство',
 'начисление',
 'сдача',
 'контрагент',
 'кассовый',
 'управленческий',
 'информация',
 'лицо',
 'налог',
 'деятельность',
 'отражение',
 'денежный',
 'большой',
 'задолженность',
 'обработка',
 'расход',
 'бюджет',
 'торговля',
 'покупатель',
 'поручение',
 'хозяйственный',
 'word']

In [23]:
cl_words4 = [x for x in words4 if x not in set(bad_words)]
cl_words4

['гость',
 'приготовление',
 'блюдо',
 'ресторан',
 'повар',
 'бесплатный',
 'чистота',
 'кухня',
 'стандарт',
 'отель',
 'технологический',
 'меню',
 'напиток',
 'желание',
 'карта',
 'уборка',
 'поддержание',
 'кафе',
 'дружный',
 'карьерный',
 'персонал',
 'бар',
 'раз',
 'официант',
 'аккуратность',
 'цех',
 'развиваться',
 'бронирование',
 'горячий',
 'час',
 'порядок',
 'готовый',
 'приглашать',
 'посуда',
 'тур',
 'человек',
 'мед',
 'речь',
 'грамотный']

In [24]:
cl_words5 = [x for x in words5 if x not in set(bad_words)]
cl_words5

['склад',
 'автомобиль',
 'водитель',
 'груз',
 'доставка',
 'перевозка',
 'категория',
 'транспортный',
 'складской',
 'водительский',
 'удостоверение',
 'транспорт',
 'хранение',
 'логистика',
 'разгрузочный',
 'вождение',
 'право',
 'отгрузка',
 'погрузка',
 'стаж',
 'товарный',
 'состояние',
 'материальный',
 'город',
 'погрузчик',
 'маршрут',
 'техника',
 'погрузочный',
 'ценность',
 'выходной',
 'кладовщик',
 'сопроводительный',
 'размещение',
 'экспедитор',
 'магазин']

In [27]:
cl_words = cl_words0+cl_words1+cl_words2+cl_words3+cl_words4+cl_words5

In [30]:
X_train['clean'] = X_train['text'].apply(lambda x: [a for a in x.split(' ') if a in cl_words])

/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
X_valid['clean'] = X_valid['text'].apply(lambda x: [a for a in x.split(' ') if a in cl_words])
X_test['clean'] = X_test['text'].apply(lambda x: [a for a in x.split(' ') if a in cl_words])

/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## TF_IDF

In [9]:
tfidf = TfidfVectorizer(max_features=10000)
tfidf.fit_transform(text_list)

<62127x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 4825154 stored elements in Compressed Sparse Row format>

# W2V

In [45]:
[' '.join(x) for x in X_train['clean'].values if x]

['осмотр больной врач диагностический лечебный мероприятие больной стандарт план больной пациент диагностический информация стандарт лечение диагностический лечебный процедура мероприятие принцип назначение пациент помощь участие формирование больной лечение пациент социальный больной состояние законодательство порядок больной информация больной выполнять диагностика состояние больной стандарт помощь диагностика план больной выполнять мероприятие выполнять лечение состояние стандарт помощь первичный участие больной лицо свой диагностика лечение больной свой деятельность пациент карта больной квалификация врач уровень средний персонал законодательство деятельность техника безопасность принцип персонал больной техника безопасность принцип поручение учреждение нормативный свой деятельность специальность социальный поддержка категория исполнительный врач стаж врач который врач врач который врач должность должность исполнительный должность должность ребенок расход расход территория размещен

In [51]:
def class_(text, cl_words):
    i = 0
    for word in text:
        if word in cl_words:
            i += 1
    return i

In [55]:
X_train['class0'] = X_train['clean'].apply(lambda x: class_(x, cl_words0))
X_train['class1'] = X_train['clean'].apply(lambda x: class_(x, cl_words1))
X_train['class2'] = X_train['clean'].apply(lambda x: class_(x, cl_words2))
X_train['class3'] = X_train['clean'].apply(lambda x: class_(x, cl_words3))
X_train['class4'] = X_train['clean'].apply(lambda x: class_(x, cl_words4))
X_train['class5'] = X_train['clean'].apply(lambda x: class_(x, cl_words5))

/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [57]:
X_valid['class0'] = X_valid['clean'].apply(lambda x: class_(x, cl_words0))
X_valid['class1'] = X_valid['clean'].apply(lambda x: class_(x, cl_words1))
X_valid['class2'] = X_valid['clean'].apply(lambda x: class_(x, cl_words2))
X_valid['class3'] = X_valid['clean'].apply(lambda x: class_(x, cl_words3))
X_valid['class4'] = X_valid['clean'].apply(lambda x: class_(x, cl_words4))
X_valid['class5'] = X_valid['clean'].apply(lambda x: class_(x, cl_words5))

/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [58]:
X_test['class0'] = X_test['clean'].apply(lambda x: class_(x, cl_words0))
X_test['class1'] = X_test['clean'].apply(lambda x: class_(x, cl_words1))
X_test['class2'] = X_test['clean'].apply(lambda x: class_(x, cl_words2))
X_test['class3'] = X_test['clean'].apply(lambda x: class_(x, cl_words3))
X_test['class4'] = X_test['clean'].apply(lambda x: class_(x, cl_words4))
X_test['class5'] = X_test['clean'].apply(lambda x: class_(x, cl_words5))

In [59]:
# Импортируем библиотеки Word2Vec
from gensim.models.word2vec import Word2Vec # Собственно модель.
from gensim.models.word2vec import LineSentence # Выравнивание текста по предложениям.
from gensim.models import KeyedVectors # Семантические вектора.

import numpy as np

In [60]:
all_texts = [x.split(' ') for x in text_list]

In [63]:
%%time

# список параметров, которые можно менять по вашему желанию
num_features = 50  # итоговая размерность вектора каждого слова
min_word_count = 2  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 4     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

model = Word2Vec(all_texts, 
                 workers=num_workers, 
                 size=num_features,
                 min_count=min_word_count, 
                 window=context,
                 sample=downsampling)

CPU times: user 2min 22s, sys: 1.66 s, total: 2min 23s
Wall time: 1min 12s


In [64]:
dct = {}

for word in model.wv.index2word:
    dct[word] = model[word]

/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [65]:
words = pd.DataFrame.from_dict(dct, orient='index')

In [66]:
words.shape

(42856, 50)

In [67]:
index2word_set = set(model.wv.index2word)

def text_to_vec(text):
    text_vec = np.zeros((model.vector_size,), dtype="float32")
    n_words = 0

    for word in text.split(' '):
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word]) 
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [68]:
text_vec = [text_to_vec(txt) for txt in text_list]

/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [69]:
train_vec = text_vec[:len(train_list)]
valid_vec = text_vec[len(train_list):len(train_list)+len(valid_list)]
test_vec = text_vec[len(train_list)+len(valid_list):len(train_list)+len(valid_list)+len(test_list)] 

In [20]:
# tfidf_m = tfidf.transform(text_list).todense()

# train_tfidf = tfidf_m[:len(train_list)]
# valid_tfidf = tfidf_m[len(train_list):len(train_list)+len(valid_list)]
# test_tfidf  = tfidf_m[len(train_list)+len(valid_list):len(train_list)+len(valid_list)+len(test_list)] 

# train_matrix = np.hstack((train_vec, train_tfidf))

# valid_matrix = np.hstack((valid_vec, valid_tfidf))

# test_matrix = np.hstack((test_vec, test_tfidf))

In [91]:
X_train_upd = X_train.reset_index(drop=True).join(pd.DataFrame(train_vec)).drop(['text', 'clean'], axis=1)

In [92]:
X_valid_upd = X_valid.reset_index(drop=True).join(pd.DataFrame(valid_vec)).drop(['text', 'clean'], axis=1)

In [93]:
X_test_upd = X_test.reset_index(drop=True).join(pd.DataFrame(test_vec)).drop(['text', 'clean'], axis=1)

## Train

In [103]:
from xgboost import XGBClassifier

In [104]:
clf = XGBClassifier(objective='multi:softmax', num_class=6)

In [111]:
param_grid = {'n_estimators':  range(150, 300, 50),
          'learning_rate': [.1],
          'max_depth': range(2, 6, 2),
          'subsample': [ .5, .6, .95, 1, .1, .3],
          'gamma': [0, .05, .1, .2, .25, 0.01, 0.02],
          'colsample_bytree': [1, .10, .50],
          'colsample_bylevel': [1, .3, .1, .5, .7, .9],
          'reg_alpha': [0, 0.1, 0.3, 0.5, 0.7, 1],
          'reg_lambda': [0, 0.1, 0.3, 0.5, 0.7, 1],
          'scale_pos_weight': [1, 10, 25, 50],
          'objective':'multi:softmax',
          'num_class':[6] }

In [114]:
X_train_upd.drop('target', axis=1, inplace=True)

In [128]:
# Задаем схему кросс-валидации
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

# Запускаем поиск гиперпараметров
hyper_search = RandomizedSearchCV(clf, param_grid, n_iter=10, scoring='accuracy', cv=cv,
                                  n_jobs=-1, refit=True, random_state=42, verbose=2)

hyper_search.fit(X_train_upd, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 14.7min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 23.5min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
          error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, num_class=6, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'n_estimators': range(150, 300, 50), 'learning_rate': [0.1], 'max_depth': range(2, 6, 2), 'subsample': [0.5, 0.6, 0.95, 1, 0.1, 0.3], 'gamma': [0, 0.05, 0.1, 0.2, 0.25, 0.01, 0.02], 'colsample_bytree': [1, 0.1, 0.5], 'colsample_bylevel': [1, 0.3, 0.1, 0.5, 0.7, 0.9], 'reg_alpha': [0, 0.1, 0.3, 0.5, 0.7, 1], 'reg_lambda': [0, 0.1, 0.3, 0.5, 0.7, 1], 'scale_pos

In [130]:
hyper_search.best_score_

0.9218635025754232

In [132]:
hyper_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=1, gamma=0.05, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=250,
       n_jobs=1, nthread=None, num_class=6, objective='multi:softprob',
       random_state=0, reg_alpha=0.1, reg_lambda=0.1, scale_pos_weight=25,
       seed=None, silent=True, subsample=0.5)

In [105]:
clf.fit(X_train_upd, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, num_class=6, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [133]:
pred = hyper_search.predict_proba(X_valid_upd)

In [134]:
y_pred = np.argmax(pred, axis=1)

In [135]:
y_pred

array([2, 0, 2, ..., 0, 5, 0])

In [136]:
accuracy_score(y_valid, y_pred)

0.9212361841399291

In [137]:
pred = hyper_search.predict_proba(X_test_upd)

In [138]:
y_pred = np.argmax(pred, axis=1)

In [139]:
test['target'] = y_pred
test[['target']].reset_index().to_csv('submission_2.csv', index=False)

In [140]:
def func(val, l=[]):
    l.append(val)
    return l

In [141]:
print(func(1))
print(func(2)) 

[1]
[1, 2]


In [70]:
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

In [97]:
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train_upd)
# Apply transform to both the training set and the test set.
train_img = scaler.transform(X_train_upd)
valid_img = scaler.transform(X_valid_upd)
test_img = scaler.transform(X_test_upd)

/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """
/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [98]:
model = LogisticRegression(solver = 'lbfgs')
model.fit(train_img, y_train)

/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [116]:
pred = model.predict(valid_img)

In [100]:
from sklearn.metrics import accuracy_score

In [102]:
accuracy_score(y_valid, pred)

0.9191973387702543

In [117]:
pred = model.predict(test_img)

In [118]:
pred

array([4, 2, 5, ..., 0, 4, 5])

In [122]:
test['target'] = pred
test[['target']].reset_index().to_csv('submission_1.csv', index=False)

In [127]:
# Задаем схему кросс-валидации
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

# Запускаем поиск гиперпараметров
hyper_search = RandomizedSearchCV(clf, param_grid, n_iter=10, scoring='accuracy', cv=cv,
                                  n_jobs=-1, refit=True, random_state=42, verbose=2)

hyper_search.fit(train_matrix, y_train)

NameError: name 'train_matrix' is not defined

## KMeans

In [15]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=10000, random_state=1)
kmeans.fit(words)

In [ ]:
crit = []

for k in range(2, 100):
    kmeans = KMeans(n_clusters=k, random_state=1).fit(X)
    crit.append(np.sqrt(kmeans.inertia_))

In [31]:
X_train

,text
17743,обязанность погрузка выгрузка груз вагон штабе...
20898,менеджер логистика обязанность ведение гос кон...
4387,обязанность согласование условие договор сопро...
8202,обязанность требоваться разный сотрудник опыт ...
9625,наш клиент российский компания разработчик фин...
...,...
1558,обязанность учет расходный договор принятие бю...
20089,приглашать становиться часть команда фудпарк н...
15114,обязанность анализ существовать бизнес процесс...
3909,игорный зона quot янтарный quot связь расширен...


21744